# Data preparation
Cleaning of the data
Splitting the data in training/testing set
Create datasets per repository
Doing anything else that needs to be done

In [64]:
# Auxiliary
import logging as log
import pymongo

import pandas as pd
from pandas import json_normalize

name = './logs/03.log'
log.basicConfig(filename=name, level=log.DEBUG, force=True, format='%(asctime)s %(message)s: ', datefmt='%Y-%m-%d %H:%M:%S')

client = pymongo.MongoClient("mongodb://localhost:27017")
db = client['JiraRepos']
coll_issues = db['01_Issues']

In [65]:
issues = coll_issues.find()
df = json_normalize(issues)

CursorNotFound: cursor id 2959562097597179212 not found, full error: {'ok': 0.0, 'errmsg': 'cursor id 2959562097597179212 not found', 'code': 43, 'codeName': 'CursorNotFound'}

In [67]:
df

,_id,expand,id,self,key,fields,changelog,repository,availability,originSprintId
0,60854a1d3c66de3fd5ed614f,"operations,versionedRepresentations,editmeta,c...",MariaDB61703,https://jira.mariadb.org/rest/api/2/issue/61703,CONJ-492,{'issuetype': {'self': 'https://jira.mariadb.o...,"{'startAt': 0, 'maxResults': 5, 'total': 5, 'h...",MariaDB,indirect,169
1,60854a1d3c66de3fd5ed615f,"operations,versionedRepresentations,editmeta,c...",MariaDB61439,https://jira.mariadb.org/rest/api/2/issue/61439,CONJ-476,{'issuetype': {'self': 'https://jira.mariadb.o...,"{'startAt': 0, 'maxResults': 7, 'total': 7, 'h...",MariaDB,indirect,169
2,60854a1d3c66de3fd5ed6189,"operations,versionedRepresentations,editmeta,c...",MariaDB60211,https://jira.mariadb.org/rest/api/2/issue/60211,CONJ-430,{'issuetype': {'self': 'https://jira.mariadb.o...,"{'startAt': 0, 'maxResults': 6, 'total': 6, 'h...",MariaDB,indirect,169
3,60854a1d3c66de3fd5ed6190,"operations,versionedRepresentations,editmeta,c...",MariaDB60091,https://jira.mariadb.org/rest/api/2/issue/60091,CONJ-422,{'issuetype': {'self': 'https://jira.mariadb.o...,"{'startAt': 0, 'maxResults': 8, 'total': 8, 'h...",MariaDB,indirect,169
4,60854a1d3c66de3fd5ed61a5,"operations,versionedRepresentations,editmeta,c...",MariaDB59295,https://jira.mariadb.org/rest/api/2/issue/59295,CONJ-400,{'issuetype': {'self': 'https://jira.mariadb.o...,"{'startAt': 0, 'maxResults': 7, 'total': 7, 'h...",MariaDB,indirect,169
...,...,...,...,...,...,...,...,...,...,...
155437,61d43ad3f4d395ee220253ce,"operations,versionedRepresentations,editmeta,c...",Spring48818,https://jira.spring.io/rest/api/2/issue/48818,DATACASS-7,{'issuetype': {'self': 'https://jira.spring.io...,"{'startAt': 0, 'maxResults': 20, 'total': 20, ...",Spring,indirect,111
155438,61d43ad3f4d395ee220253d1,"operations,versionedRepresentations,editmeta,c...",Spring48815,https://jira.spring.io/rest/api/2/issue/48815,DATACASS-4,{'issuetype': {'self': 'https://jira.spring.io...,"{'startAt': 0, 'maxResults': 17, 'total': 17, ...",Spring,indirect,124
155439,61d43ad3f4d395ee22025428,"operations,versionedRepresentations,editmeta,c...",Spring48108,https://jira.spring.io/rest/api/2/issue/48108,BATCHADM-155,{'issuetype': {'self': 'https://jira.spring.io...,"{'startAt': 0, 'maxResults': 5, 'total': 5, 'h...",Spring,indirect,2
155440,61d43ad3f4d395ee2202542c,"operations,versionedRepresentations,editmeta,c...",Spring47409,https://jira.spring.io/rest/api/2/issue/47409,BATCHADM-151,{'issuetype': {'self': 'https://jira.spring.io...,"{'startAt': 0, 'maxResults': 7, 'total': 7, 'h...",Spring,indirect,2


In [68]:
df.to_pickle('df.pickle')